In [1]:
import polars as pl
from polars import col as c
from datetime import datetime
import pandas as pd

## 2025-02-13 closer look at PERMNO 22798

1. Delisting: it seems to have a single delisting event on 2024-09-18 according to `/wrds/crsp/sasdata/a_stock_v2/stkdelists.sas7bdat`
2. That is beyond the date-range under question, 2023 mid-December. 
3. Of the all the columns from the daily_stock_v2 table, wonder which ones to rely on for the information of whether a stock is "dropped from the CRSP universe"?
   1. The `Del*` columns seem like a fit, but a closer look shows that the value of this column continues to be the same for much of the latter half of 2023, thus difficult to pinpoint which day.


In [2]:
df_two = pl.from_pandas(pd.read_stata('/home/hzhang/data/wrds/crsp_two_2023_cases.dta.gz'))

In [3]:
d = pl.from_pandas(pd.read_sas('/home/hzhang/data/wrds/stkdelists.sas7bdat', encoding="latin-1"))

In [4]:
d.filter(c('PERMNO').eq(22798))

PERMNO,DelistingDt,DelDtPrc,DelDtPrcFlg,DelActionType,DelStatusType,DelReasonType,DelPaymentType,DelPERMNO,DelPERMCO,DelRet,DelRetMissType,DelNextDt,DelNextPrc,DelNextPrcFlg,DelAmtDt,DelDivAmt,DelDisType,DelDlyDt
f64,datetime[ms],f64,str,str,str,str,str,f64,f64,f64,str,datetime[ms],f64,str,datetime[ms],f64,str,datetime[ms]
22798.0,2024-09-18 00:00:00,0.78,"""TR""","""GDR""","""VCL""","""FING""","""PRCF""",0.0,0.0,-0.812821,"""NA""",2024-09-19 00:00:00,0.146,"""DP""",2024-09-19 00:00:00,0.0,"""NO""",2024-09-19 00:00:00


In [5]:
with pl.Config(tbl_cols=-1, tbl_rows=100):
    display(df_two.filter(c('PERMNO').eq(22798) & c('DlyCalDt').is_between(datetime.fromisoformat('2023-12-10'), datetime.fromisoformat('2023-12-20'))))

PERMNO,SecInfoStartDt,SecInfoEndDt,SecurityBegDt,SecurityEndDt,SecurityHdrFlg,HdrCUSIP,HdrCUSIP9,CUSIP,CUSIP9,PrimaryExch,ConditionalType,ExchangeTier,TradingStatusFlg,SecurityNm,ShareClass,USIncFlg,IssuerType,SecurityType,SecuritySubType,ShareType,SecurityActiveFlg,DelActionType,DelStatusType,DelReasonType,DelPaymentType,Ticker,TradingSymbol,PERMCO,SICCD,NAICS,ICBIndustry,NASDCompno,NASDIssuno,IssuerNm,YYYYMMDD,DlyCalDt,DlyDelFlg,DlyPrc,DlyPrcFlg,DlyCap,DlyCapFlg,DlyPrevPrc,DlyPrevPrcFlg,DlyPrevDt,DlyPrevCap,DlyPrevCapFlg,DlyRet,DlyRetx,DlyRetI,DlyRetMissFlg,DlyRetDurFlg,DlyOrdDivAmt,DlyNonOrdDivAmt,DlyFacPrc,DlyDistRetFlg,DlyVol,DlyClose,DlyLow,DlyHigh,DlyBid,DlyAsk,DlyOpen,DlyNumTrd,DlyMMCnt,DlyPrcVol,ShrStartDt,ShrEndDt,ShrOut,ShrSource,ShrFacType,ShrAdrFlg,DisExDt,DisSeqNbr,DisOrdinaryFlg,DisType,DisFreqType,DisPaymentType,DisDetailType,DisTaxType,DisOrigCurType,DisDivAmt,DisFacPr,DisFacShr,DisDeclareDt,DisRecordDt,DisPayDt,DisPERMNO,DisPERMCO,vwretd,vwretx,ewretd,ewretx,sprtrn
f64,datetime[ns],datetime[ns],datetime[ns],datetime[ns],str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,str,str,f64,f64,str,f64,datetime[ns],str,f64,str,f64,str,f64,str,datetime[ns],f64,str,f64,f64,f64,str,str,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,datetime[ns],datetime[ns],f64,str,str,str,datetime[ns],f64,str,str,str,str,str,str,str,f64,f64,f64,datetime[ns],datetime[ns],datetime[ns],f64,f64,f64,f64,f64,f64,f64
22798.0,2023-10-27 00:00:00,2023-12-14 00:00:00,2022-02-16 00:00:00,2024-09-18 00:00:00,"""N""","""83204U60""","""83204U608""","""83204U50""","""83204U509""","""Q""","""RW""","""S""","""A""","""SMART FOR LIFE INC; COM NONE; …","""""","""Y""","""CORP""","""EQTY""","""COM""","""NS""","""Y""","""GDR""","""VCL""","""FING""","""PRCF""","""SMFL""","""SMFL""",59195.0,9999.0,"""325412""","""NOAVAIL""",6.0079691e7,86651.0,"""SMART FOR LIFE INC""",2.0231211e7,2023-12-11 00:00:00,"""N""",0.8141,"""TR""",648.8377,"""BP""",1.03,"""TR""",2023-12-08 00:00:00,820.91,"""PB""",-0.209612,-0.209612,0.0,"""NA""","""D3""",0.0,0.0,1.0,"""NO""",431588.0,0.8141,0.76,1.0,0.8099,0.81,1.0,1576.0,24.0,351355.7908,2023-11-17 00:00:00,2023-12-14 00:00:00,797.0,"""OBS""","""NA""","""N""",null,null,"""""","""""","""""","""""","""""","""""","""""",null,null,null,null,null,null,null,null,0.00369,0.003684,-0.004576,-0.004604,0.003924
22798.0,2023-10-27 00:00:00,2023-12-14 00:00:00,2022-02-16 00:00:00,2024-09-18 00:00:00,"""N""","""83204U60""","""83204U608""","""83204U50""","""83204U509""","""Q""","""RW""","""S""","""A""","""SMART FOR LIFE INC; COM NONE; …","""""","""Y""","""CORP""","""EQTY""","""COM""","""NS""","""Y""","""GDR""","""VCL""","""FING""","""PRCF""","""SMFL""","""SMFL""",59195.0,9999.0,"""325412""","""NOAVAIL""",6.0079691e7,86651.0,"""SMART FOR LIFE INC""",2.0231212e7,2023-12-12 00:00:00,"""N""",0.7603,"""TR""",605.9591,"""BP""",0.8141,"""TR""",2023-12-11 00:00:00,648.8377,"""PB""",-0.066085,-0.066085,0.0,"""NA""","""D1""",0.0,0.0,1.0,"""NO""",116693.0,0.7603,0.76,0.8001,0.7601,0.78,0.7897,541.0,24.0,88721.6879,2023-11-17 00:00:00,2023-12-14 00:00:00,797.0,"""OBS""","""NA""","""N""",null,null,"""""","""""","""""","""""","""""","""""","""""",null,null,null,null,null,null,null,null,0.003133,0.00309,-0.00421,-0.004254,0.004599
22798.0,2023-10-27 00:00:00,2023-12-14 00:00:00,2022-02-16 00:00:00,2024-09-18 00:00:00,"""N""","""83204U60""","""83204U608""","""83204U50""","""83204U509""","""Q""","""RW""","""S""","""A""","""SMART FOR LIFE INC; COM NONE; …","""""","""Y""","""CORP""","""EQTY""","""COM""","""NS""","""Y""","""GDR""","""VCL""","""FING""","""PRCF""","""SMFL""","""SMFL""",59195.0,9999.0,"""325412""","""NOAVAIL""",6.0079691e7,86651.0,"""SMART FOR LIFE INC""",2.0231213e7,2023-12-13 00:00:00,"""N""",0.791,"""TR""",630.427,"""BP""",0.7603,"""TR""",2023-12-12 00:00:00,605.9591,"""PB""",0.040379,0.040379,0.0,"""NA""","""D1""",0.0,0.0,1.0,"""NO""",279671.0,0.791,0.745,0.864,0.79,0.8,0.76,1098.0,20.0,22121

## Pre - 2025-02-13
## Reproducing `DlyPrevDt`
## Sorted dataframe
Note that the dataframe is already sorted by `['DlyCalDt', 'PERMNO']`

In [6]:
df_crsp = pl.read_parquet('/home/hzhang/data/wrds/crsp_prevdt.parq')

In [7]:
## assert non NaN in any float columns
assert not any(df_crsp.select(pl.col(pl.Float64,pl.Float32)).select(pl.all().is_nan().any()).row(0)),  "There are NaN values in at least one float column."

### Full logic

`prev_cal_dt` is the *last*, *non-null* DlyPrc, up to 10 periods ago:
1. *last*: so the present value is filled forward, and then `shift`ed
2. *non-null*: thus the fill forward is only performed when `DlyPrc` is non-null (the `when` clause)
3. up to 10 periods: thus there is a limit on fill forward periods, of 10-1 periods due to the `shift`


In [8]:
max_missing_period = 10
df = (df_crsp
      .unique(subset=['DlyCalDt', 'PERMNO'], keep='first', maintain_order=True) # get rid of multiple rows of dividends
      .with_columns(
          prev_cal_dt = pl.when(c('DlyPrc').is_not_null()).then(c('DlyCalDt')).forward_fill(limit=max_missing_period-1).shift().over('PERMNO'))) #  

## Compare to CRSP value
1. Boundary effects: if the ground truth `DlyPrevDt` goes beyond dataframe's start_date, then it's OK not to match.
2. Only care when `DlyPrc` is not null
3. Use `ne_missing` so that `null` patterns match as well

I'm left with a very small number of rows of mismatch, out of 67M rows. 

Note that a simple fix by setting `prev_cal_dt` to null when smaller than `SecInfoStartDt` results in many more mismatches elsewhere.

In [9]:
min_date = df['DlyCalDt'].min()
with pl.Config(tbl_cols=-1):
    display(df.filter(
        (c('DlyPrevDt').ge(min_date) | c('DlyPrevDt').is_null()) &
        c("DlyPrc").is_not_null() &
        c('DlyPrevDt').ne_missing(c('prev_cal_dt')))
            )

PERMNO,SecInfoStartDt,SecInfoEndDt,HdrCUSIP,SecurityActiveFlg,Ticker,PERMCO,DlyCalDt,DlyPrc,DlyPrevDt,prev_cal_dt
f64,datetime[ns],datetime[ns],str,str,str,f64,datetime[ns],f64,datetime[ns],datetime[ns]
11178.0,1994-08-03 00:00:00,1994-09-08 00:00:00,"""90462840""","""Y""","""UNRC""",9026.0,1994-08-03 00:00:00,1.875,null,1994-07-20 00:00:00
81014.0,1995-06-26 00:00:00,1997-04-22 00:00:00,"""89323B20""","""Y""","""TSRG""",13370.0,1995-06-26 00:00:00,1.0,null,1995-06-16 00:00:00
76942.0,1995-09-18 00:00:00,1996-04-01 00:00:00,"""29087M30""","""Y""","""EMRL""",11091.0,1995-09-18 00:00:00,0.21875,null,1995-09-13 00:00:00
10155.0,1996-08-22 00:00:00,1997-02-06 00:00:00,"""28485330""","""Y""","""ELGT""",8079.0,1996-08-22 00:00:00,1.125,null,1996-08-09 00:00:00
88459.0,2008-04-28 00:00:00,2008-06-26 00:00:00,"""69511V20""","""Y""","""PACT""",37950.0,2008-04-28 00:00:00,0.9988,null,2008-04-24 00:00:00
…,…,…,…,…,…,…,…,…,…,…
14675.0,2020-06-10 00:00:00,2020-06-18 00:00:00,"""87876P20""","""Y""","""TGEN""",54873.0,2020-06-10 00:00:00,0.75,null,2020-06-08 00:00:00
21018.0,2023-10-18 00:00:00,2023-11-05 00:00:00,"""02369M10""","""Y""","""AMAO""",58077.0,2023-10-18 00:00:00,10.895,null,2023-10-13 00:00:00
22798.0,2023-12-18 00:00:00,2024-04-21 00:00:00,"""83204U60""","""Y""","""SMFL""",59195.0,2023-12-18 00:00:00,0.6589,null,2023-12-14 00:00:00


In [10]:
df.shape

(67644099, 11)